In [3]:
%matplotlib inline
# ---- settings ----
import json
Settings = json.load(open('../settings.txt'))
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys
sys.path.insert(0, '../'); sys.path.insert(0, '../samples')
import paf_loader
import numpy as np
import cv2
from cselect import color as cs
from time import time
import mvpose.data.kth_football2 as kth
from mvpose import pose
import paf_loader

from mvpose.algorithm import settings

params = settings.get_settings(scale_to_mm=1000)

colors = cs.lincolor(16).astype('float32')/255
FRAME = 2 # 85
data_root = Settings['data_root']
seq1_zipname = 'player1sequence2.zip'
seq1_dir = 'Sequence 2'
# seq1_zipname = 'player2sequence1.zip'
# seq1_dir = 'Sequence 1'
player = 1
Im, Calib, pos2d, pos3d = kth.get(
    data_root, seq1_zipname, seq1_dir, FRAME, player=player)


from mvpose.algorithm.candidate_selection import project_human_to_2d

heatmaps, pafs = paf_loader.load_confidence_map_and_paf(
    'kth' + seq1_zipname[0:-4], Im, FRAME, dir='/home/julian/Temp')
_start = time()
Debug, detections = pose.estimate(Calib, heatmaps, pafs,
                                  settings=params, debug=True)
_end = time()
print('elapsed', _end - _start)

Humans = kth.transform3d_from_mscoco(detections)
print('#Humans', len(Humans))

fig = plt.figure(figsize=(16,10))
for cid, im in enumerate(Im):
    ax = fig.add_subplot(1,3,cid+1); ax.axis('off')
    ax.imshow(im, alpha=0.5)
    joints = pos2d[:,:,cid]
    for jid, (x,y) in enumerate(joints):
        ax.scatter(x,y,color=colors[jid], marker='*')

    for pid, human in enumerate(Humans):
        human2d = project_human_to_2d(human, Calib[cid])
        for jid, p in enumerate(human2d):
            if p is not None:
                ax.scatter(p[0], p[1], color=colors[jid], marker='s')

plt.show()

downloading...  http://www.csc.kth.se/cvap/cvg/MultiViewFootballData/player1sequence2.zip


KeyboardInterrupt: 

In [ ]:
import mvpose.data.skeleton_augmentation as ska
model_path = '../data/model_poseprediction.h5'
gen = ska.LimbGenerator(model_path, params.scale_to_mm)

detections = gen.apply(detections)
Humans2 = kth.transform3d_from_mscoco(detections)

fig = plt.figure(figsize=(16,10))

colors = cs.lincolor(5).astype('float32')/255

for cid, im in enumerate(Im):
    ax = fig.add_subplot(1,3,cid+1); ax.axis('off')
    ax.imshow(im, alpha=0.5)
    
    for pid, human in enumerate(Humans2):
        kth.draw_limbs2d(ax, human, Calib[cid], color='red', print_length=False)
    
    for pid, human in enumerate(Humans):
        kth.draw_limbs2d(ax, human, Calib[cid], color="green", print_length=False)
    
        
plt.show()

In [3]:
from mvpose.evaluation import pcp

model_path = '../data/model_poseprediction.h5'
gen = ska.LimbGenerator(model_path, params.scale_to_mm)
use_heuristic = True

larms = []
uarms = []
llegs = []
ulegs = []
pelvis = []
shoulder = []
head = []
all_parts = []

larms_ex = []
uarms_ex = []
llegs_ex = []
ulegs_ex = []
pelvis_ex = []
shoulder_ex = []
head_ex = []
all_parts_ex = []

def extract_best(pos3d, Humans):
    larms = 0
    uarms = 0
    ulegs = 0
    llegs = 0
    all_parts = 0
    pelvis = 0
    
    for h in Humans:
        r = pcp.evaluate(pos3d, Humans[0], alpha)
        if all_parts < r.all_parts:
            larms = r.lower_arms
            uarms = r.upper_arms
            ulegs = r.upper_legs
            llegs = r.lower_legs
            pelvis = r.pelvis
            all_parts = r.all_parts
    
    return {
        "larms": larms,
        "uarms": uarms,
        "ulegs": ulegs,
        "llegs": llegs,
        "pelvis": pelvis,
        "all_parts": all_parts
    }
    

for frame in range(0, 2*11, 2):
    _start = time()
    Im, Calib, pos2d, pos3d = kth.get(
        data_root, seq1_zipname, seq1_dir, frame, player=player)
    heatmaps, pafs = paf_loader.load_confidence_map_and_paf(
        'kth' + seq1_zipname[0:-4], Im, frame, dir='/home/julian/Temp')
    
    if use_heuristic:
        detections = pose.estimate_heuristic(
            Calib, heatmaps, pafs, settings=params)
    else:
        detections = pose.estimate(
            Calib, heatmaps, pafs, settings=params)
    
    alpha = 0.5
    
    # ~~~~~ no adding of limbs ~~~~~
    Humans = kth.transform3d_from_mscoco(detections)
    result = extract_best(pos3d, Humans)
    larms.append(result['larms'])
    uarms.append(result['uarms'])
    llegs.append(result['llegs'])
    ulegs.append(result['ulegs'])
    pelvis.append(result['pelvis'])
    all_parts.append(result['all_parts'])
    
    # ~~~~~~ adding limbs if needed ~~~~~~
    detections = gen.apply(detections)
    Humans = kth.transform3d_from_mscoco(detections)
    result = extract_best(pos3d, Humans)
    larms_ex.append(result['larms'])
    uarms_ex.append(result['uarms'])
    llegs_ex.append(result['llegs'])
    ulegs_ex.append(result['ulegs'])
    pelvis_ex.append(result['pelvis'])
    all_parts_ex.append(result['all_parts'])

    
#     del Humans
#     del detections
    
    _end = time()
    print('frame ' + str(frame) + ', elapsed', _end - _start)

frame 0, elapsed 5.1809961795806885
frame 2, elapsed 3.74214506149292
frame 4, elapsed 4.761949777603149
frame 6, elapsed 4.798750638961792


KeyboardInterrupt: 

In [ ]:
print('+++ No limb heuristic +++')
print('lower arms:\t', np.mean(larms))
print('upper arms:\t', np.mean(uarms))
print('lower legs:\t', np.mean(llegs))
print('upper legs:\t', np.mean(ulegs))
print('pelvis:\t\t', np.mean(pelvis))
print('shoulder:\t', np.mean(shoulder))
print('head:\t\t', np.mean(head))
print('all parts:\t', np.mean(all_parts))

print('+++ With limb heuristic +++')
print('lower arms:\t', np.mean(larms_ex))
print('upper arms:\t', np.mean(uarms_ex))
print('lower legs:\t', np.mean(llegs_ex))
print('upper legs:\t', np.mean(ulegs_ex))
print('pelvis:\t\t', np.mean(pelvis_ex))
print('shoulder:\t', np.mean(shoulder_ex))
print('head:\t\t', np.mean(head_ex))
print('all parts:\t', np.mean(all_parts_ex))

In [ ]:
# this has to be done as my computer always crashes when I run longer runs
It1 = {
    'llegs': [1.0, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 0, 0.5, 1.0, 1.0],
    'ulegs': [1.0, 0.5, 0.5, 1.0, 1.0, 1.0, 1.0, 0, 1.0, 1.0, 1.0],
    'uarms': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 0, 0.5, 1.0, 1.0],
    'larms': [1.0, 1.0, 1.0, 0.5, 0.5, 1.0, 0.5, 0, 0.5, 1.0, 1.0],
    'all_parts': [0.9166666666666667, 0.75,0.6666666666666666,0.9166666666666667,0.9166666666666667,
         0.9166666666666667,0.5833333333333333,0,0.75,1.0, 1.0]
}

It1_ex = {
    'llegs': [1.0, 0.5, 0.5, 1.0, 1.0, 1.0, 1.0, 0, 1.0, 1.0, 1.0],
    'ulegs': [1.0, 0.5, 0.5, 1.0, 1.0, 1.0, 1.0, 0, 1.0, 1.0, 1.0],
    'uarms': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0, 1.0, 1.0, 1.0],
    'larms': [1.0, 1.0, 1.0, 0.5, 0.5, 1.0, 0.5, 0, 0.5, 1.0, 1.0],
    'all_parts': [0.9166666666666667, 0.75, 0.75, 0.9166666666666667, 0.9166666666666667, 1.0,
         0.9166666666666667, 0, 0.9166666666666667, 1.0, 1.0]
}

It2 = {
    'llegs': [0.5, 1.0, 1.0, 0.5,  0,  0, 0.5, 0.5, 0.5, 1.0],
    'ulegs': [0.5, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0],
    'uarms': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5],
    'larms': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5],
    'all_parts': [0.6666666666666666, 1.0, 1.0, 0.9166666666666667, 0.75, 0.75, 0.8333333333333334,
         0.6666666666666666, 0.6666666666666666, 0.6666666666666666]
}

It2_ex = {
    'llegs': [0.5, 1.0, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0],
    'ulegs': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0],
    'uarms': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
    'larms': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5],
    'all_parts': [0.8333333333333334, 1.0, 1.0, 1.0, 0.9166666666666667, 0.9166666666666667,
                  0.9166666666666667, 0.9166666666666667, 0.8333333333333334, 0.9166666666666667]
}

In [ ]:
for key in ['llegs', 'ulegs', 'uarms', 'larms', 'all_parts']:
    total = It1[key] + It2[key]
    total_ex = It1_ex[key] + It2_ex[key]
    print(key + '(raw output):\t\t', np.mean(total))
    print(key + '(post-proc):\t\t', np.mean(total_ex))
    print('')

print('\navg(raw):', (0.6190476190476191 + 0.7380952380952381 + 0.8809523809523809 + 0.8333333333333334)/4)
print('\navg(pproc):', (0.7619047619047619 + 0.8809523809523809 + 0.9523809523809523 + 0.8333333333333334)/4)